In [1]:
from scenarios_config import get_scenario

# Get the modified gravity scenario with exponent 2.03
scenario = get_scenario('periastron', '3.1 M, 0.18 M, Elliptical, Single Orbit', 
                       row_wise=True,
                       max_observations_total=100, 
                       max_observations_per_request=10)

# Access the binary simulation object
binary_sim = scenario.binary_sim

In [2]:
print(binary_sim.row_wise_prompt)


You are tasked with solving the following physics problem related to a binary star system. You are provided observations of each star's position over time, (t,x,y,z), in units of seconds and meters.

### Problem Description
Determine the periastron distance of the binary star system.
You must provide your answer in units of m.

### Additional Instructions
To complete this task, you have access to the following tools and data:
1. An observational tool called `Observe` that allows you observe the system at
specific times of your choosing.
2. A code interpreter that can execute Python code.

When using `Observe`:
1. The `times_requested` parameter should be a list that can contain any values in the time window [0.0, 2.34e+08] seconds. You cannot request negative times. The upper limit for the time window was chosen to gurantee that the problem is solvable with an appropriate sampling of observations using the total observational budget.
2. You can observe the system at any time within the

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
max_time = 2.34e+08
initial_times = list(np.linspace(0, max_time, 10))
print(binary_sim.observe_row(initial_times, maximum_observations_per_request=10))


Observations added to row_wise_results.df. You have 90 observations remaining in your total budget. 


/Users/./Desktop/AST1500/physics-agent-bench/generalscenarios/Binary.py:287: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.row_wise_results.df = pd.concat([self.row_wise_results.df, new_rows], ignore_index=True)


In [4]:
def get_separation(df):
    # Calculate separation distance between stars
    r = np.sqrt((df['star2_x'] - df['star1_x'])**2 + 
                (df['star2_y'] - df['star1_y'])**2 + 
                (df['star2_z'] - df['star1_z'])**2)
    return r

# Calculate and print separation distances
separations = get_separation(binary_sim.row_wise_results.df)
print("Times (s) and separation distances (m):")
for t, sep in zip(binary_sim.row_wise_results.df['time'], separations):
    print(f"t={t:.2e} s, r={sep:.2e} m")
# Get indices of 3 lowest separations
lowest_indices = np.argsort(separations)[:3]

# Print the 3 lowest separations and their times
for idx in lowest_indices:
    sep = separations[idx]
    t = binary_sim.row_wise_results.df['time'].iloc[idx]
    print(f"Separation: {sep:.2e} m at t={t:.2e} s")

Times (s) and separation distances (m):
t=0.00e+00 s, r=1.14e+11 m
t=2.60e+07 s, r=9.14e+11 m
t=5.20e+07 s, r=1.30e+12 m
t=7.80e+07 s, r=1.52e+12 m
t=1.04e+08 s, r=1.62e+12 m
t=1.30e+08 s, r=1.61e+12 m
t=1.56e+08 s, r=1.50e+12 m
t=1.82e+08 s, r=1.27e+12 m
t=2.08e+08 s, r=8.55e+11 m
t=2.34e+08 s, r=1.36e+11 m
Separation: 1.14e+11 m at t=0.00e+00 s
Separation: 1.36e+11 m at t=2.34e+08 s
Separation: 8.55e+11 m at t=2.08e+08 s


It looks like the min seperation occurs either around t = 0 or t = 2.34e+08 seconds. I will look in both places.

In [5]:
extent = max_time/10
times_1 = list(np.linspace(0, extent, 10))
times_2 = list(np.linspace(max_time-extent, max_time, 10))
print(binary_sim.observe_row(times_1, maximum_observations_per_request=10))
print(binary_sim.observe_row(times_2, maximum_observations_per_request=10))


Observations added to row_wise_results.df. You have 80 observations remaining in your total budget. 

Observations added to row_wise_results.df. You have 70 observations remaining in your total budget. 


In [6]:
# Calculate and print separation distances
separations = get_separation(binary_sim.row_wise_results.df)
# Get indices of 3 lowest separations
lowest_indices = np.argsort(separations)[:3]

# Print the 3 lowest separations and their times
for idx in lowest_indices:
    sep = separations[idx]
    t = binary_sim.row_wise_results.df['time'].iloc[idx]
    print(f"Separation: {sep:.2e} m at t={t:.2e} s")

Separation: 9.39e+10 m at t=2.31e+08 s
Separation: 1.14e+11 m at t=0.00e+00 s
Separation: 1.14e+11 m at t=0.00e+00 s


Now it looks like the lowest seperation is closer to 2.31e+08 seconds. I'll make 10 more observations around there with a smaller time interval.

In [7]:
extent = max_time/50
times = list(np.linspace(2.31e+08-extent, 2.31e+08+extent, 10))
print(binary_sim.observe_row(times, maximum_observations_per_request=10))


Observations added to row_wise_results.df. You have 60 observations remaining in your total budget. 


In [8]:
# Calculate and print separation distances
separations = get_separation(binary_sim.row_wise_results.df)
# Get indices of 3 lowest separations
lowest_indices = np.argsort(separations)[:3]

# Print the 3 lowest separations and their times
for idx in lowest_indices:
    sep = separations[idx]
    t = binary_sim.row_wise_results.df['time'].iloc[idx]
    print(f"Separation: {sep:.2e} m at t={t:.2e} s")

Separation: 6.20e+10 m at t=2.33e+08 s
Separation: 8.73e+10 m at t=2.32e+08 s
Separation: 9.39e+10 m at t=2.31e+08 s


Let's use the rest of my observations around 2.33e+08 seconds.

In [9]:
extent = max_time/50
times = list(np.linspace(2.33e+08-extent, 2.33e+08+extent, 10))
print(binary_sim.observe_row(times, maximum_observations_per_request=10))
# Calculate and print separation distances
separations = get_separation(binary_sim.row_wise_results.df)
# Get indices of 3 lowest separations
lowest_indices = np.argsort(separations)[:3]

# Print the 3 lowest separations and their times
for idx in lowest_indices:
    sep = separations[idx]
    t = binary_sim.row_wise_results.df['time'].iloc[idx]
    print(f"Separation: {sep:.2e} m at t={t:.2e} s")


Observations added to row_wise_results.df. You have 50 observations remaining in your total budget. 
Note: Some requested times exceeded the maximum time of 233615056.0132853. For these times, None values were inserted for positions.
Separation: 6.03e+10 m at t=2.32e+08 s
Separation: 6.20e+10 m at t=2.33e+08 s
Separation: 8.73e+10 m at t=2.32e+08 s


/Users/./mambaforge/envs/physicsbench/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: The behavior of Series.argsort in the presence of NA values is deprecated. In a future version, NA values will be ordered last instead of set to -1.
  return bound(*args, **kwds)


In [12]:
min_sep = separations[lowest_indices[0]]
print(f"Min Separation: {min_sep:.2e} m")
print(f"True Answer: {scenario.true_answer():.2e} m")
print(f"{np.abs(min_sep - scenario.true_answer())/scenario.true_answer() * 100:.2f}% off from true answer")

Min Separation: 6.03e+10 m
True Answer: 5.91e+10 m
2.14% off from true answer
